# Simple ML - Testing simple models#

What it does?
- Reads the csv file created by the Statistics Parser
- Drops the string columns and keeps only the columns with the grades
- Tests a model for each course-column using the LeaveOneOut method and the XGBoost Regressor
- Prints the results in terms of MAE and RMSE

In [13]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
data = pd.read_csv("csd.csv")
display(data)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
data.describe()

In [ ]:
data.info()

### Rename Columns ###

In [15]:
data.rename(columns={"Ηλικία":"age","Φύλο":"gender","Επέλεξα τη σχολή μου διότι:":"reason",
                     "Κατά μέσο όρο την εβδομάδα, διαβάζω:":"study_time",
                    "Μέσα στο εξάμηνο, παρακαλουθώ:":"lectures",
                     "Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;":"private",
                    "Μετά το πτυχίο, θα ήθελα να ακολουθήσω:":"postgraduate",
                    "Ποιο από τα παρακάτω ισχύει;":"roomates",
                    "Η σχολή απέχει από το σπίτι μου:":"distance",
                    "Ασχολούμαι εβδομαδιαία με:":"hobbies"},inplace=True)
display(data)

,age,gender,reason,study_time,lectures,private,postgraduate,roomates,distance,hobbies,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


### Specify imports

In [16]:
from sklearn.model_selection import LeaveOneOut
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
import math

import warnings #needed for this type of classifier
warnings.simplefilter(action='ignore', category=Warning)


In [17]:
# Select course columns
course_columns = data.columns[10:]
print(course_columns)

Index(['NCO-01-01', 'NCO-01-02', 'NCO-01-03', 'NCO-01-04', 'NCO-01-05',
       'NCO-02-01', 'NNA-06-01', 'NNA-06-02', 'NNA-07-01', 'NCO-02-02',
       'NNA-07-02', 'NCO-02-03', 'NCO-02-04', 'NCO-02-05', 'NNA-07-03',
       'NNA-07-04', 'NNA-07-05', 'NNA-07-06', 'NNA-07-08', 'NCO-03-01',
       'NCO-03-02', 'NCO-03-03', 'NCO-03-04', 'NCO-03-05', 'NCO-04-01',
       'NCO-04-02', 'NCO-04-03', 'NCO-04-04', 'NCO-04-05', 'NCO-05-01',
       'NNA-08-03', 'NCO-05-02', 'NCO-05-03', 'NCO-05-04', 'NCO-05-05',
       'NET-06-01', 'NIS-06-01', 'NET-06-02', 'NET-06-03', 'NIS-06-02',
       'NET-07-01', 'NIS-06-03', 'NET-07-02', 'NET-07-03', 'NIS-06-04',
       'NGE-06-01', 'NGE-06-02', 'NGE-06-03', 'NDM-06-01', 'NIS-06-05',
       'NDM-06-02', 'NIS-06-06', 'NET-08-02', 'NIS-07-01', 'NET-08-01',
       'NIS-07-02', 'NIS-07-04', 'NIS-07-05', 'NDM-07-01', 'NGE-07-01',
       'NIS-07-06', 'NDM-07-02', 'NIS-08-01', 'NDM-07-04', 'NIS-08-02',
       'NDM-08-01', 'NDM-08-02', 'NIS-08-03', 'NDM-08-03', 'NIS-

### Categorical Values Handling - Function ###

In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MultiLabelBinarizer


def handle_categorical(data):
    # Categorical values that maintain the scaling properties "study_time","lectures","postgraduate","distance"
    # Study_time
    data["study_time"] = data["study_time"].replace({"0 - 2 ώρες":0.0,"2 - 5 ώρες":1.0,"> 5 ώρες":2.0})
    data["study_time"] = pd.to_numeric(data["study_time"])

    # Lectures
    data["lectures"] = data["lectures"].replace({"Λιγότερες από τις μισές διαλέξεις":0.0,
                                                 "Περίπου τις μισές διαλέξεις":1.0,
                                                 "Παραπάνω από τις μισές διαλέξεις":2.0,
                                                 "Όλες τις διαλέξεις":3.0})
    data["lectures"] = pd.to_numeric(data["lectures"])

    # Postgraduate
    data["postgraduate"] = data["postgraduate"].replace({"Τίποτα από τα δύο":0.0,
                                                         "Μεταπτυχιακές Σπούδες":1.0,
                                                         "Διδακτορικές Σπουδές":2.0})
    data["postgraduate"] = pd.to_numeric(data["postgraduate"])

    # Distance
    data["distance"] = data["distance"].replace({"< 10 λεπτά":0.0,
                                                 "10 - 25 λεπτά":1.0,
                                                 "25 - 45 λεπτά":2.0,
                                                 "> 45 λεπτά":3.0})
    data["distance"] = pd.to_numeric(data["distance"])

    # Gender
    data["gender"] = data["gender"].replace({"Κορίτσι":1,"Αγόρι":0})
    data["gender"] = pd.to_numeric(data["gender"])

    # private
    data["private"] = data["private"].replace({"Ναι":0,
                                              "Όχι":1})
    data["private"] = pd.to_numeric(data["private"])
    
    
    # One-hot encoder columns (only roomates)

    ohe_columns = ["roomates"]

    full_pipeline = ColumnTransformer([
        ('one_hot',OneHotEncoder(),ohe_columns)
    ])

    roomates = full_pipeline.fit_transform(data)
    # Concat with  data with roomates
    roomates_df = pd.DataFrame(roomates.toarray(),columns=['family','alone','friend','siblings'],dtype=np.int8)
    data_updated = pd.concat([data.drop("roomates",axis=1),roomates_df],axis=1)
    
    
    
    # Convert string cell with multiple values to list
    acceptable_hobbies = ["Σειρές / Ταινίες","Αθλητισμό","Video Games","Ξένη γλώσσα","Εθελοντισμός"]
    for student in range(0,data.shape[0]):
        data['reason'][student] = data['reason'][student].split(", ") #There is a space after each comma
        # Hobbies transformation

        hobbies_list = data['hobbies'][student].split(", ")
        for i in range(0,len(hobbies_list)):
            if hobbies_list[i] not in acceptable_hobbies:
                hobbies_list[i] = "Άλλο"
        data['hobbies'][student] = hobbies_list
        
        
        # Multilabel Binarizer

        mlb = MultiLabelBinarizer()
        hobbies = mlb.fit_transform(data['hobbies'])
#         print(mlb.classes_)
        reasons = mlb.fit_transform(data['reason'])
#         print(mlb.classes_)

    # Convert to dataframe and concat
    hobbies_df = pd.DataFrame(hobbies,columns=["vgames","other","sports","volunteer","languange","movies"])
    data_updated = pd.concat([data_updated.drop(["hobbies"],axis=1),hobbies_df],axis=1)

    reasons_df = pd.DataFrame(reasons,columns=["quality","choice","subject","parents","career"])
    data_updated = pd.concat([data_updated.drop(["reason"],axis=1),reasons_df],axis=1)
    
    return data_updated



### Handle Categorical ###

In [19]:
data = handle_categorical(data)

In [20]:
data

,age,gender,study_time,lectures,private,postgraduate,distance,NCO-01-01,NCO-01-02,NCO-01-03,...,other,sports,volunteer,languange,movies,quality,choice,subject,parents,career
0,19,1,0.0,3.0,1,1.0,2.0,10.0,10.0,10.0,...,0,0,0,0,1,1,0,1,0,1
1,19,0,2.0,2.0,1,2.0,0.0,6.0,10.0,7.0,...,0,1,0,0,1,0,0,1,0,1
2,22,1,1.0,2.0,1,1.0,2.0,6.0,7.0,5.0,...,0,1,1,0,1,0,0,1,0,1
3,20,0,2.0,1.0,1,1.0,0.0,7.0,10.0,8.0,...,0,1,0,0,1,0,0,1,0,0
4,20,0,0.0,3.0,1,1.0,0.0,10.0,10.0,10.0,...,0,0,1,0,0,0,0,1,0,0
5,22,0,1.0,1.0,1,2.0,2.0,7.0,10.0,6.0,...,0,0,1,0,1,0,0,1,0,0
6,19,0,0.0,1.0,1,1.0,1.0,6.0,9.0,9.0,...,0,1,0,0,0,0,0,1,0,1
7,20,0,0.0,3.0,1,1.0,0.0,8.0,10.0,10.0,...,0,0,0,1,1,0,0,1,0,1
8,20,0,2.0,3.0,1,1.0,0.0,4.0,9.0,4.0,...,1,0,0,0,1,1,0,1,0,1
9,22,0,2.0,3.0,1,1.0,1.0,7.0,10.0,10.0,...,0,1,0,0,1,0,0,1,0,0


### Courses + Age ###

In [ ]:
# Let's drop categorical columns for now (Courses and age)
# print(len(data.columns))
selected_columns = [0] + list(range(10,len(data.columns)))
print(selected_columns)

### Only Courses ###

In [ ]:
# Using only the courses
selected_columns = list(range(10,len(data.columns)))
print(selected_columns)

### Only Characteristics ###

In [ ]:
# Using only the user characteristics
selected_columns = list(range(0,10))
print(selected_columns)

In [21]:
# Select only those columns
data_selected = data


In [22]:
sum_error = 0
sum_squared_error = 0
for course_selected in course_columns: #For each course
    print(course_selected)
    errors = []
    sq_errors = []
    if course_selected in data_selected:
        X = data_selected.drop(course_selected,axis=1,inplace=False)
    y = data.loc[:,course_selected]
    loo = LeaveOneOut()
    xgb = XGBRegressor(objective = 'reg:squarederror')
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        xgb.fit(X_train,y_train)
        predictions = xgb.predict(X_test)
        errors.append(mean_absolute_error(y_test,predictions))
        sq_errors.append(mean_squared_error(y_test,predictions))
    
    sum_error += np.mean(errors)
    sum_squared_error += np.mean(sq_errors)**(1/2)
    print("MAE:" + str(np.mean(errors)))
    print("RMSE:" + str((np.mean(sq_errors))**(1/2)))

print("Mean MAE:" + str( sum_error / (len(data_selected.columns) - 1)))
print("Mean RMSE:" + str( sum_squared_error / (len(data_selected.columns) - 1)))


NCO-01-01
MAE:1.7631670181141343
RMSE:2.3294892829066356
NCO-01-02
MAE:1.2477352854817412
RMSE:1.9985510011517666
NCO-01-03
MAE:1.7206173178761504
RMSE:2.4857520600200496
NCO-01-04
MAE:2.180733995382176
RMSE:2.741175883328181
NCO-01-05
MAE:2.5584914157556935
RMSE:3.454568662430673
NCO-02-01
MAE:2.336504284725633
RMSE:3.0865749722996316
NNA-06-01
MAE:5.1140785217285156e-05
RMSE:5.1140785217285156e-05
NNA-06-02
MAE:0.4631159582803416
RMSE:1.7120076475139463
NNA-07-01
MAE:5.1140785217285156e-05
RMSE:5.1140785217285156e-05
NCO-02-02
MAE:2.37025187181872
RMSE:3.2065526629744805
NNA-07-02
MAE:5.1140785217285156e-05
RMSE:5.1140785217285156e-05
NCO-02-03
MAE:2.5362151181975077
RMSE:3.4003371492695997
NCO-02-04
MAE:2.293138401452885
RMSE:2.9564152214305497
NCO-02-05
MAE:1.7053530964740486
RMSE:2.3543898798144998
NNA-07-03
MAE:5.1140785217285156e-05
RMSE:5.1140785217285156e-05
NNA-07-04
MAE:5.1140785217285156e-05
RMSE:5.1140785217285156e-05
NNA-07-05
MAE:5.1140785217285156e-05
RMSE:5.11407852172

### Corellation Table ###

In [ ]:

corrmat = data_selected.corr(method="pearson")
top_corr_features = corrmat.index
plt.figure(figsize=(35, 35))
g = sns.heatmap(data_selected[top_corr_features].corr(), annot=True, cmap="RdYlGn")
plt.show()